In [ ]:
import pydap.client
import requests
import numpy
import configparser

# Part 1: Parsing DMRs
the client has the following functions to open downloaded files
- ```open_file()```
- ```open_dods_file()```
- ```open_dmr_file()```
- ```open_dap_file()```

In [5]:
fname = 'data/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.dmr'
dataset = pydap.client.open_dmr_file(fname)
dataset['sea_ice_fraction']

<BaseType with data <pydap.parsers.dmr.DummyData object at 0x7fb6b0488550>>

# Part 2: Loading dap and DMR files

## Example 1

In [2]:
fname = 'data/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_subset.dap'
dataset = pydap.client.open_dap_file(fname)
dataset

<DatasetType with children 'sea_ice_fraction'>

In [3]:
dataset['sea_ice_fraction']

<BaseType with data array([[[ 98,  98],
        [ 98,  98],
        [ 98,  98],
        ...,
        [100, 100],
        [100, 100],
        [100, 100]]], dtype=int8)>

In [4]:
dataset['sea_ice_fraction'].attributes

{'long_name': 'sea ice area fraction',
 'standard_name': 'sea_ice_area_fraction',
 '_FillValue': '-128',
 'add_offset': '0.',
 'scale_factor': '0.01',
 'valid_min': '0',
 'valid_max': '100',
 'source': 'EUMETSAT OSI-SAF, copyright EUMETSAT',
 'comment': 'ice fraction is a dimensionless quantity between 0 and 1; it has been interpolated by a nearest neighbor approach.',
 'coordinates': 'lon lat',
 'checksum': array([3847985684], dtype=uint32)}

## Example 2

In [5]:
fname = 'data/coads_climatology.nc_full.dap'
dataset = pydap.client.open_dap_file(fname)
dataset

<DatasetType with children 'COADSX', 'COADSY', 'TIME', 'SST', 'AIRT', 'UWND', 'VWND'>

In [6]:
dataset['SST'].array[1, 1:10, 1:10]

<BaseType with data array([[-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34],
       [-1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34, -1.e+34,
        -1.e+34, -1.e+34]], dtype=float32)>

In [4]:
fname = 'data/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf.dap'
pydap.client.open_dap_file(fname)

<DatasetType with children 'sst_qual_b', 'Number_of_records', 'Number_of_samples_per_record', 'Latitude', 'Longitude'>

# Part 3: Accessing OPeNDAP server

## Example 1: Previously inaccesible through pydap
as posted by Alberto Torres in the GH issues https://github.com/pydap/pydap/issues/226#issuecomment-1198288875

The example demonstrates the ability to access

- Data requiring an earthdata loging
- Data that is served up exclusively through DAP4
- Data containing 8, 16, 32 bit integers and 32 bit floats

### Creating a session 
Note:
The username and password are loaded from the file `user.config`. Edit this file accordingly or enter your credentials here directly

In [7]:
config = configparser.ConfigParser()
config.read('user.config')
username = config['user']['user']
password = config['user']['pwd']

In [8]:
class SessionEarthData(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
                    redirect_parsed.hostname != self.AUTH_HOST and \
                    original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']
        return

session = SessionEarthData(username=username, password=password)

### Opening the dataset URL to build a dataset
- ```pydap.client.open_url()``` downloads DMR, pareses it, and builds a dataset from it. I.e. interpreting :
    - data types including endianess
    - shapes
    - hierarchy (groups)
    - relations (maps) of variables and dimensions
    - variable attributes
- No data is downloaded; rather, 'DummyData' of the according type and shpae is inserted into the dataset
- Using the dap 4 protocol is specified using either
    - Using the schema 'dap4://' in the url (canonical)
    - Specifing the 'schema' kwarg in the function call: ```pydap.client.open_url(url, protocol='dap4', **kwargs)```.

In [9]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'opendap.earthdata.nasa.gov'
path = '/collections/C1996881146-POCLOUD/granules/'
dataset = '20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'
dap4_url

'dap4://opendap.earthdata.nasa.gov/collections/C1996881146-POCLOUD/granules/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1'

#### Accessing with DAP2 
We first demnonstrate that accessing the dataset via dap2 fails:

In [10]:
pydap.client.open_url(dap2_url, session=session)

KeyError: 'int8'

#### Accessing via DAP4

In [14]:
pydap_ds = pydap.client.open_url(dap4_url, session=session)
#pydap_ds = pydap.client.open_url(dap2_url, session=session, protocol='dap4')
pydap_ds

<DatasetType with children 'mask', 'analysed_sst', 'lon', 'time', 'sea_ice_fraction', 'dt_1km_data', 'lat', 'analysis_error', 'sst_anomaly'>

#### Inspecting the dataset

In [15]:
pydap_ds._dict

OrderedDict([('mask',
              <BaseType with data Dap4BaseProxy('http://opendap.earthdata.nasa.gov/collections/C1996881146-POCLOUD/granules/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1', 'mask', dtype('int8'), (1, 17999, 36000), (slice(None, None, None), slice(None, None, None), slice(None, None, None)))>),
             ('analysed_sst',
              <BaseType with data Dap4BaseProxy('http://opendap.earthdata.nasa.gov/collections/C1996881146-POCLOUD/granules/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1', 'analysed_sst', dtype('>i2'), (1, 17999, 36000), (slice(None, None, None), slice(None, None, None), slice(None, None, None)))>),
             ('lon',
              <BaseType with data Dap4BaseProxy('http://opendap.earthdata.nasa.gov/collections/C1996881146-POCLOUD/granules/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1', 'lon', dtype('>f4'), (36000,), (slice(None, None, None),))>),
             ('time',
              <BaseType with data Dap4Ba

### Accessing the data
We can access the variables attributes. Note that the checksum is part of the data response, not the DMR. 

In [16]:
pydap_ds['sea_ice_fraction'].attributes

{'long_name': 'sea ice area fraction',
 'standard_name': 'sea_ice_area_fraction',
 '_FillValue': -128,
 'add_offset': 0.0,
 'scale_factor': 0.01,
 'valid_min': 0,
 'valid_max': 100,
 'source': 'EUMETSAT OSI-SAF, copyright EUMETSAT',
 'comment': 'ice fraction is a dimensionless quantity between 0 and 1; it has been interpolated by a nearest neighbor approach.',
 'coordinates': 'lon lat'}

#### Lazy subsetting
- pydap uses numpy-stype fancy indexing; i.e ```[start:stop:step]```
- opendap uses fancy indexing of following style ```[start:step:stop]```
- pydap translates between those styles

In [17]:
variable = pydap_ds['sea_ice_fraction'][0, 1700:1799:10, 1800:1900:10]

In [18]:
variable.data

array([[[95, 95, 95, 95, 95, 95, 95, 95, 95, 95],
        [95, 95, 95, 95, 95, 94, 95, 95, 94, 94],
        [95, 95, 95, 95, 94, 94, 94, 94, 94, 94],
        [95, 95, 94, 94, 94, 94, 94, 94, 94, 94],
        [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
        [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
        [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
        [94, 94, 94, 94, 93, 93, 93, 93, 93, 94],
        [94, 94, 94, 93, 93, 93, 93, 93, 93, 93],
        [94, 94, 94, 94, 94, 93, 93, 93, 93, 93]]], dtype=int8)

After accessing the data, the variables attributes now contain the checksum

In [19]:
variable.attributes

{'long_name': 'sea ice area fraction',
 'standard_name': 'sea_ice_area_fraction',
 '_FillValue': -128,
 'add_offset': 0.0,
 'scale_factor': 0.01,
 'valid_min': 0,
 'valid_max': 100,
 'source': 'EUMETSAT OSI-SAF, copyright EUMETSAT',
 'comment': 'ice fraction is a dimensionless quantity between 0 and 1; it has been interpolated by a nearest neighbor approach.',
 'coordinates': 'lon lat',
 'checksum': array([3760107333], dtype=uint32)}

#### Masking values

In [20]:
fill_value = int(variable.attributes['_FillValue'])
mask = (variable.data==fill_value)
numpy.ma.array(variable.data, mask=mask)

masked_array(
  data=[[[95, 95, 95, 95, 95, 95, 95, 95, 95, 95],
         [95, 95, 95, 95, 95, 94, 95, 95, 94, 94],
         [95, 95, 95, 95, 94, 94, 94, 94, 94, 94],
         [95, 95, 94, 94, 94, 94, 94, 94, 94, 94],
         [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
         [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
         [94, 94, 94, 94, 94, 94, 94, 94, 94, 94],
         [94, 94, 94, 94, 93, 93, 93, 93, 93, 94],
         [94, 94, 94, 93, 93, 93, 93, 93, 93, 93],
         [94, 94, 94, 94, 94, 93, 93, 93, 93, 93]]],
  mask=[[[False, False, False, False, False, False, False, False, False,
          False],
         [False, False, False, False, False, False, False, False, False,
          False],
         [False, False, False, False, False, False, False, False, False,
          False],
         [False, False, False, False, False, False, False, False, False,
          False],
         [False, False, False, False, False, False, False, False, False,
          False],
         [False, Fa

## Example 2: Groups and 64 bit floats
- Dataset containing groups
- Dataset containing 64 bit floats 

In [21]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'test.opendap.org:8080'
path = '/opendap/dmrpp_test_files/'
dataset = 'ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'

### DAP2

In [22]:
ds_dap2 = pydap.client.open_url(dap2_url)
ds_dap2

<DatasetType with children '/gt1r/bckgrd_atlas/bckgrd_int_height', '/gt1r/bckgrd_atlas/delta_time', '/gt1r/heights/lat_ph', '/gt1r/heights/delta_time'>

In [23]:
ds_dap2._dict

OrderedDict([('/gt1r/bckgrd_atlas/bckgrd_int_height',
              <BaseType with data BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/bckgrd_atlas/bckgrd_int_height', dtype('>f4'), (89490,), (slice(None, None, None),))>),
             ('/gt1r/bckgrd_atlas/delta_time',
              <BaseType with data BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/bckgrd_atlas/delta_time', dtype('>f8'), (89490,), (slice(None, None, None),))>),
             ('/gt1r/heights/lat_ph',
              <BaseType with data BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/heights/lat_ph', dtype('>f8'), (4102521,), (slice(None, None, None),))>),
             ('/gt1r/heights/delta_time',
              <BaseType with data BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/

In [24]:
ds_dap2['/gt1r/bckgrd_atlas/delta_time']

<BaseType with data BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/bckgrd_atlas/delta_time', dtype('>f8'), (89490,), (slice(None, None, None),))>

In [25]:
ds_dap2['/gt1r/bckgrd_atlas/delta_time'][0:10].data

array([31197611.26648983, 31197611.27148983, 31197611.27648983,
       31197611.28148983, 31197611.28648982, 31197611.29148982,
       31197611.29648982, 31197611.30148982, 31197611.30648983,
       31197611.31148983])

### DAP4

In [26]:
ds_dap4 = pydap.client.open_url(dap4_url)
ds_dap4

<DatasetType with children '/gt1r/bckgrd_atlas/bckgrd_int_height', '/gt1r/bckgrd_atlas/delta_time', '/gt1r/heights/lat_ph', '/gt1r/heights/delta_time'>

In [27]:
ds_dap4._dict

OrderedDict([('/gt1r/bckgrd_atlas/bckgrd_int_height',
              <BaseType with data Dap4BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/bckgrd_atlas/bckgrd_int_height', dtype('>f4'), (89490,), (slice(None, None, None),))>),
             ('/gt1r/bckgrd_atlas/delta_time',
              <BaseType with data Dap4BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/bckgrd_atlas/delta_time', dtype('>f8'), (89490,), (slice(None, None, None),))>),
             ('/gt1r/heights/lat_ph',
              <BaseType with data Dap4BaseProxy('http://test.opendap.org:8080/opendap/dmrpp_test_files/ATL03_20181228015957_13810110_003_01.2var.h5.dmrpp', '/gt1r/heights/lat_ph', dtype('>f8'), (4102521,), (slice(None, None, None),))>),
             ('/gt1r/heights/delta_time',
              <BaseType with data Dap4BaseProxy('http://test.opendap.org:8080/opendap/d

In [29]:
data = ds_dap4['/gt1r/bckgrd_atlas/delta_time'][0:10]
data

<BaseType with data array([31197611.26648983, 31197611.27148983, 31197611.27648983,
       31197611.28148983, 31197611.28648982, 31197611.29148982,
       31197611.29648982, 31197611.30148982, 31197611.30648983,
       31197611.31148983])>

In [30]:
data.attributes

{'description': 'Elapsed GPS Seconds from the ATLAS SDP GPS Epoch, referenced to the start of the 50-shot sum. This is based on every fiftieth laser fire time, which leads to a very close alignment with major frame boundaries (+/- 1 shot). The ATLAS Standard Data Products (SDP) epoch offset is defined within /ancillary_data/atlas_sdp_gps_epoch as the number of GPS seconds between the GPS epoch (1980-01-06T00:00:00.000000Z UTC) and the ATLAS SDP epoch. By adding the offset contained within atlas_sdp_gps_epoch to delta time parameters, the time in gps_seconds relative to the GPS epoch can be computed.',
 'long_name': 'Time at the start of ATLAS 50-shot sum',
 'source': 'ATL02',
 'standard_name': 'time',
 'units': 'seconds since 2018-01-01',
 'checksum': array([4139645269], dtype=uint32)}

## Example 3: Maps/Grids

In [31]:
dap2_schema = 'http'
dap4_schema = 'dap4'

host = 'opendap.earthdata.nasa.gov'
path = '/hyrax/data/nc/'
dataset = 'coads_climatology.nc'

dap2_url = f'{dap2_schema}://{host}{path}{dataset}'
dap4_url = f'{dap4_schema}://{host}{path}{dataset}'

In [32]:
ds_dap2 = pydap.client.open_url(dap2_url, session=session)
ds_dap4 = pydap.client.open_url(dap4_url, session=session)

In [33]:
data_dap2 = ds_dap2['SST'][0:1:1, 40:42:1, 0:20:2]

In [34]:
data_dap2.data

[array([[[-9.9999998e+33, -9.9999998e+33, -9.9999998e+33, -9.9999998e+33,
          -9.9999998e+33,  2.8480732e+01,  2.8533415e+01,  2.8464815e+01,
           2.8058332e+01,  2.7882812e+01],
         [-9.9999998e+33, -9.9999998e+33, -9.9999998e+33, -9.9999998e+33,
          -9.9999998e+33,  2.8278717e+01,  2.8698605e+01,  2.8708000e+01,
           2.7853157e+01,  2.7896786e+01]]], dtype=float32),
 array([366.]),
 array([-9., -7.]),
 array([21., 25., 29., 33., 37., 41., 45., 49., 53., 57.])]

In [35]:
data_dap4 = ds_dap4['SST'][0:1:1, 40:42:1, 0:20:2]

In [36]:
data_dap4.data[:]

[array([[[-9.9999998e+33, -9.9999998e+33, -9.9999998e+33, -9.9999998e+33,
          -9.9999998e+33,  2.8480732e+01,  2.8533415e+01,  2.8464815e+01,
           2.8058332e+01,  2.7882812e+01],
         [-9.9999998e+33, -9.9999998e+33, -9.9999998e+33, -9.9999998e+33,
          -9.9999998e+33,  2.8278717e+01,  2.8698605e+01,  2.8708000e+01,
           2.7853157e+01,  2.7896786e+01]]], dtype=float32),
 array([366.]),
 array([-9., -7.]),
 array([21., 25., 29., 33., 37., 41., 45., 49., 53., 57.])]

## Example 4 MOD05

In [37]:
url = 'dap4://test.opendap.org/opendap/hyrax/data/stare/MOD05_L2.A2019336.2315.061.2019337071952.hdf'

In [38]:
ds = pydap.client.open_url(url)

In [39]:
ds['Water_Vapor_Infrared']

<BaseType with data Dap4BaseProxy('http://test.opendap.org/opendap/hyrax/data/stare/MOD05_L2.A2019336.2315.061.2019337071952.hdf', 'Water_Vapor_Infrared', dtype('>i2'), (406, 270), (slice(None, None, None), slice(None, None, None)))>

## Example 5 MY1DQND

In [40]:
url = 'http://test.opendap.org/opendap/MODIS/MOOA/MY1DQN.004/2004.12.31/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf'

In [41]:
ds = pydap.client.open_url(url)

In [42]:
ds['sst_qual_b']

<BaseType with data BaseProxy('http://test.opendap.org/opendap/MODIS/MOOA/MY1DQN.004/2004.12.31/MY1DQND1.sst.ADD2005001.040.2006011070802.hdf', 'sst_qual_b', dtype('uint8'), (180, 360), (slice(None, None, None), slice(None, None, None)))>